# Brain MRI Comparative Analysis - Example Notebook

This notebook demonstrates how to use the DeepPrep analysis pipeline for comparing brain MRI scans between healthy individuals and patients with neurological disorders.

## Contents
1. Setup and Import Libraries
2. Generate Synthetic Data
3. Preprocessing MRI Scans
4. Feature Extraction
5. Comparative Analysis
6. Visualization
7. Interpretation of Results

## 1. Setup and Import Libraries

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

from deepprep_analysis import (
    MRIPreprocessor,
    FeatureExtractor,
    ComparativeAnalyzer,
    Visualizer
)
from deepprep_analysis.data_generator import SyntheticMRIGenerator

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

## 2. Generate Synthetic Data

We'll generate synthetic brain MRI scans for demonstration purposes.

In [ ]:
# Initialize generator
generator = SyntheticMRIGenerator(image_shape=(64, 64, 64))

# Generate dataset
print("Generating synthetic MRI data...")
generator.generate_dataset(
    n_healthy=15,
    n_disorder=15,
    output_dir='../data'
)
print("Data generation complete!")

In [ ]:
# Visualize sample healthy and disorder MRI
healthy_mri = generator.generate_healthy_mri(subject_id=0)
disorder_mri = generator.generate_disorder_mri(subject_id=0)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Healthy brain slices
slices = [20, 32, 44]
for i, slice_idx in enumerate(slices):
    axes[0, i].imshow(healthy_mri[:, :, slice_idx].T, cmap='gray', origin='lower')
    axes[0, i].set_title(f'Healthy - Slice {slice_idx}')
    axes[0, i].axis('off')

# Disorder brain slices
for i, slice_idx in enumerate(slices):
    axes[1, i].imshow(disorder_mri[:, :, slice_idx].T, cmap='gray', origin='lower')
    axes[1, i].set_title(f'Disorder - Slice {slice_idx}')
    axes[1, i].axis('off')

plt.suptitle('Sample MRI Scans: Healthy vs Disorder', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. Preprocessing MRI Scans

Load and preprocess the MRI scans.

In [ ]:
# Get file paths
data_dir = Path('../data')
healthy_files = list((data_dir / 'healthy').glob('*.nii.gz'))
disorder_files = list((data_dir / 'disorder').glob('*.nii.gz'))

print(f"Found {len(healthy_files)} healthy scans")
print(f"Found {len(disorder_files)} disorder scans")

In [ ]:
# Initialize preprocessor
preprocessor = MRIPreprocessor()

# Preprocess healthy scans
print("Preprocessing healthy scans...")
healthy_preprocessed = preprocessor.preprocess_dataset(
    [str(f) for f in healthy_files],
    normalize=True,
    remove_bg=True,
    skull_strip=True,
    normalization_method='z-score'
)

# Preprocess disorder scans
print("Preprocessing disorder scans...")
disorder_preprocessed = preprocessor.preprocess_dataset(
    [str(f) for f in disorder_files],
    normalize=True,
    remove_bg=True,
    skull_strip=True,
    normalization_method='z-score'
)

print(f"\nPreprocessing complete!")
print(f"Healthy: {len(healthy_preprocessed)} scans")
print(f"Disorder: {len(disorder_preprocessed)} scans")

In [ ]:
# Compare preprocessing statistics
healthy_stats = [scan['stats'] for scan in healthy_preprocessed]
disorder_stats = [scan['stats'] for scan in disorder_preprocessed]

healthy_stats_df = pd.DataFrame(healthy_stats)
disorder_stats_df = pd.DataFrame(disorder_stats)

print("\nHealthy Group - Preprocessing Statistics:")
print(healthy_stats_df[['mean_intensity', 'std_intensity', 'non_zero_voxels']].describe())

print("\nDisorder Group - Preprocessing Statistics:")
print(disorder_stats_df[['mean_intensity', 'std_intensity', 'non_zero_voxels']].describe())

## 4. Feature Extraction

Extract quantitative features from preprocessed MRI scans.

In [ ]:
# Initialize feature extractor
feature_extractor = FeatureExtractor(n_pca_components=20)

# Extract features
print("Extracting features from healthy scans...")
healthy_features = feature_extractor.extract_features_from_dataset(healthy_preprocessed)

print("Extracting features from disorder scans...")
disorder_features = feature_extractor.extract_features_from_dataset(disorder_preprocessed)

print(f"\nFeature extraction complete!")
print(f"Number of features per scan: {len(healthy_features[0]) - 1}")  # -1 for filepath

In [ ]:
# Display sample features
healthy_df = pd.DataFrame(healthy_features)
disorder_df = pd.DataFrame(disorder_features)

print("Sample features from healthy group:")
print(healthy_df.head())

print("\nFeature statistics:")
print(healthy_df.describe().iloc[:, :8])  # Show first 8 columns

## 5. Comparative Analysis

Perform statistical comparisons between the two groups.

In [ ]:
# Initialize analyzer
analyzer = ComparativeAnalyzer(alpha=0.05)

# Perform complete analysis
print("Performing comparative analysis...")
analysis_results = analyzer.analyze_group_differences(
    healthy_features,
    disorder_features
)

print("\nAnalysis complete!")

In [ ]:
# Display key results
t_tests = analysis_results['t_tests']
classification = analysis_results['classification']

print("=" * 80)
print("STATISTICAL ANALYSIS RESULTS")
print("=" * 80)
print(f"\nSample Sizes:")
print(f"  Healthy: {analysis_results['n_healthy']}")
print(f"  Disorder: {analysis_results['n_disorder']}")

print(f"\nFeature Analysis:")
print(f"  Total features: {len(t_tests)}")
print(f"  Significant features (p < 0.05): {t_tests['significant'].sum()}")
print(f"  Percentage significant: {t_tests['significant'].sum() / len(t_tests) * 100:.1f}%")

print(f"\nClassification Performance:")
print(f"  Accuracy: {classification['cv_accuracy_mean']:.3f} ± {classification['cv_accuracy_std']:.3f}")
print(f"  Number of features used: {classification['n_features']}")

In [ ]:
# Top discriminative features
print("\nTop 10 Most Discriminative Features:")
print("=" * 80)
top_features = t_tests.nsmallest(10, 'p_value')
for idx, row in top_features.iterrows():
    print(f"{idx+1}. {row['feature']}")
    print(f"   p-value: {row['p_value']:.6f}")
    print(f"   Cohen's d: {row['cohens_d']:.3f}")
    print(f"   Mean difference: {row['mean_difference']:.3f}")
    print(f"   Significant: {'Yes' if row['significant'] else 'No'}")
    print()

## 6. Visualization

Generate visualizations to understand the results.

In [ ]:
# Initialize visualizer
visualizer = Visualizer(output_dir='../results')

# Plot feature distributions for top features
top_feature_names = t_tests.nsmallest(6, 'p_value')['feature'].tolist()
visualizer.plot_feature_distributions(
    analysis_results['data'],
    top_feature_names
)
plt.show()

In [ ]:
# Plot t-test results
visualizer.plot_t_test_results(analysis_results['t_tests'], top_n=15)
plt.show()

In [ ]:
# Plot feature importance
visualizer.plot_feature_importance(
    analysis_results['classification']['feature_importances'],
    top_n=15
)
plt.show()

In [ ]:
# Create comprehensive summary report
visualizer.create_summary_report(analysis_results)
plt.show()

## 7. Interpretation of Results

Let's interpret the key findings:

In [ ]:
# Analyze feature categories
t_tests_sorted = t_tests.sort_values('p_value')

# Categorize features
volume_features = [f for f in t_tests_sorted['feature'] if 'volume' in f.lower()]
intensity_features = [f for f in t_tests_sorted['feature'] if 'intensity' in f.lower()]
morphology_features = [f for f in t_tests_sorted['feature'] if any(
    x in f.lower() for x in ['sphericity', 'elongation', 'compactness', 'surface']
)]
texture_features = [f for f in t_tests_sorted['feature'] if any(
    x in f.lower() for x in ['entropy', 'energy', 'homogeneity', 'contrast']
)]
regional_features = [f for f in t_tests_sorted['feature'] if 'region' in f.lower()]

print("Feature Category Analysis:")
print("=" * 80)
print(f"Volume features: {len(volume_features)}")
print(f"Intensity features: {len(intensity_features)}")
print(f"Morphology features: {len(morphology_features)}")
print(f"Texture features: {len(texture_features)}")
print(f"Regional features: {len(regional_features)}")

In [ ]:
# Create summary table
summary_data = {
    'Metric': [
        'Sample Size (Healthy)',
        'Sample Size (Disorder)',
        'Total Features',
        'Significant Features',
        'Classification Accuracy',
        'Most Important Feature',
        'Largest Effect Size'
    ],
    'Value': [
        analysis_results['n_healthy'],
        analysis_results['n_disorder'],
        len(t_tests),
        t_tests['significant'].sum(),
        f"{classification['cv_accuracy_mean']:.3f} ± {classification['cv_accuracy_std']:.3f}",
        classification['feature_importances'].iloc[0]['feature'],
        t_tests.loc[t_tests['cohens_d'].abs().idxmax(), 'feature']
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\nSummary Table:")
print("=" * 80)
print(summary_df.to_string(index=False))

## Conclusion

This notebook demonstrated:

1. **Data Generation**: Created synthetic brain MRI data with realistic characteristics
2. **Preprocessing**: Applied normalization, background removal, and skull stripping
3. **Feature Extraction**: Computed volume, intensity, texture, morphological, and regional features
4. **Statistical Analysis**: Performed t-tests and effect size calculations
5. **Classification**: Used Random Forest to distinguish between groups
6. **Visualization**: Created comprehensive visualizations of results

### Key Findings:

- Multiple features show significant differences between healthy and disorder groups
- Classification accuracy indicates good separability between groups
- Volume, intensity, and morphological features are particularly discriminative
- Regional analysis reveals spatially localized differences

### Next Steps:

- Apply to real MRI data
- Explore additional preprocessing techniques
- Test different feature extraction methods
- Investigate deep learning approaches
- Perform longitudinal analysis